## Reexecução

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
os.environ['SNORKELDB'] = 'postgres:///lzirondi'
from snorkel import SnorkelSession
session = SnorkelSession()

from snorkel.models import candidate_subclass
relation = candidate_subclass('Causa', ['Substância', 'Evento_Adverso'])

## File Paths & Magic Functions

In [38]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

inPath = '/home/lzirondi/Github/AlexandreMartins/'
outPath = '/home/lzirondi/Github/snorkel/Scripts/Datasets'
corpusPath = outPath + '/Processed/Tweets Anotados.tsv'
LIWCpath = '/home/lzirondi/Github/snorkel/Scripts/'



#Calculando o número de linhas no corpus
with open(corpusPath) as tsv:
    s = tsv.readlines()
    n_docs = len(s)
    s = None

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Nova execução

In [2]:
#Fazendo o link simbólico para o acesso ao banco de dados e definindo o banco como o PSQL
import os
os.system("ln -s /var/run/postgresql/.s.PGSQL.5432 /tmp/.s.PGSQL.5432")
os.environ['SNORKELDB'] = 'postgres:///lzirondi'

#O import do SnorkelSession SEMPRE vai ter que estar após definir o BD nas variaveis de ambiente do SO, ao contrário ele defaulta para SQLite
from snorkel import SnorkelSession
session = SnorkelSession()

## Gerando Sentenças

In [3]:
from snorkel.parser import TSVDocPreprocessor
doc_preprocessor = TSVDocPreprocessor(corpusPath, max_docs=n_docs)

#

from snorkel.parser.spacy_parser import Spacy
from snorkel.parser import CorpusParser

corpus_parser = CorpusParser(parser=Spacy())

%time corpus_parser.apply(doc_preprocessor, count=n_docs, parallelism=16)

from snorkel.models import Document, Sentence

print("Documents:", session.query(Document).count())
print("Sentences:", session.query(Sentence).count())

Clearing existing...


  0%|          | 0/5984 [00:00<?, ?it/s]

Running UDF...


100%|██████████| 5984/5984 [01:25<00:00, 69.83it/s] 

CPU times: user 2.97 s, sys: 824 ms, total: 3.79 s
Wall time: 1min 27s
Documents: 5984
Sentences: 6031


## Preparando os Dicionários

In [42]:
import Preprocessing as prep


util = prep.Util(inPath, outPath, LIWCpath, True)


substancias = util.subsToList()
eventos = util.eventosToList()

Arquivo Tweets Anotados.txt copiado com sucesso.
Arquivo Eventos.txt copiado com sucesso.
Arquivo Substâncias2-br-gazette.txt copiado com sucesso.
Arquivo Remédios2-br-gazette.txt copiado com sucesso.
Arquivo Substâncias-br-gazette.txt copiado com sucesso.
Arquivo Remédios-br-gazette.txt copiado com sucesso.
Arquivo EventosAdversos-gazette.txt copiado com sucesso.
Tsv do Corpus criado com sucesso.
Dicionário de Substâncias gerado com sucesso.
Dicionário de Eventos gerado com sucesso.
Dicionário LIWC de Causas gerado com sucesso.
Dicionário LIWC de Verbos gerado com sucesso.
Dicionário LIWC de Causas & Verbos gerado com sucesso.


## Gerando a Classe de Candidatos

In [5]:
#Separar
from snorkel.models import candidate_subclass

relation = candidate_subclass('Causa', ['Substância', 'Evento_Adverso'])

## Extraindo Candidatos do Corpus

In [6]:
from snorkel.candidates import Ngrams, CandidateExtractor
from snorkel.matchers import DictionaryMatch

ngrams = Ngrams(n_max=7)
substancia_matcher = DictionaryMatch(d=substancias, longest_match_only=True, ignore_case=True)
eventos_matcher = DictionaryMatch(d=eventos, longest_match_only=True, ignore_case=True)
cand_extractor = CandidateExtractor(relation, [ngrams, ngrams], [substancia_matcher, eventos_matcher], symmetric_relations=True)

from snorkel.models import Document

docs = session.query(Document).order_by(Document.name).all()

train_sents = set()
dev_sents   = set()
test_sents  = set()

for i, doc in enumerate(docs):
    for s in doc.sentences:
        if i % 10 == 8:
            dev_sents.add(s)
        elif i % 10 == 9:
            test_sents.add(s)
        else:
            train_sents.add(s)
                
#

%time
for i, sents in enumerate([train_sents, dev_sents, test_sents]):
    cand_extractor.apply(sents, split=i, parallelism = 16)
    print("Number of candidates:", session.query(relation).filter(relation.split == i).count())

  0%|          | 0/4825 [00:00<?, ?it/s]

CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 11 µs
Clearing existing...
Running UDF...


  0%|          | 0/603 [00:00<?, ?it/s]

Number of candidates: 6398
Clearing existing...
Running UDF...


  0%|          | 0/603 [00:00<?, ?it/s]

Number of candidates: 765
Clearing existing...
Running UDF...


100%|██████████| 603/603 [00:13<00:00, 45.99it/s]

Number of candidates: 762


# Execução Incompleta

## Gerando um .tsv dos candidatos

In [8]:
cands = session.query(relation).filter(relation.split == 0).all()

f = open('/home/lzirondi/Github/snorkel/Scripts/Datasets/Processed/Candidatos.tsv', 'w', encoding='UTF-8')
f.write('Sentença\t Span_1 \t Span_2 \n')

for candidate in cands:   
    
    parent = candidate.get_parent()
    parent = parent._asdict()
    parent = parent.get('text')
    parent = parent.replace('\n', '')
    
    
    contexts = candidate.get_contexts()
    span_list = [contexts[0].get_span(), contexts[1].get_span()]   
    
    
    f.write(parent + '\t' + span_list[0] + '\t' + span_list[1] + '\n' )
    
    
f.close()

## Random Stuff

In [ ]:
#par = candi.get_parent()
#dict1 = par._asdict()

#contexts = candi.get_contexts()

#spans = [contexts[0], contexts[1]]

#a = contexts[0]

#for candidate in cands:
#    print(candidate.get_contexts()[0].get_span())
#print(a.get_span())


#object_methods = [method_name for method_name in dir(a) if callable(getattr(a, method_name))]

#print(object_methods)
#cands = session.query(causas).filter(causas.split == 0).all()

#cands2 = session.query(causas).filter(causas.split==0).all()
relation = candidate_subclass('Causa', ['Substância', 'Evento_Adverso'])

print(session.query(causas).filter(causas.split==0).all())

#print(cands)
#print(cands[527])

#print(cands2[527])


from snorkel.lf_helpers import (
    get_left_tokens, get_right_tokens, get_between_tokens,
    get_text_between, get_tagged_text,
)



In [ ]:
#print(type(parent))
#print(type(contexts_list[0]))
#print(type(contexts_list[0]))
#print(type(cands[0]))
    

#print(cands[0].get_cids())
#print(type(cands[0].get_contexts()))
#print(cands[0].get_parent())

#object_methods = [method_name for method_name in dir(cands[0]) if callable(getattr(cands[0], method_name))]

#print(object_methods)

#a = cands[0].get_contexts()

#object_methods = [method_name for method_name in dir(contexts[0]) if callable(getattr(contexts[0], method_name))]

#print(object_methods)

#print(type(a))

#f = open('/home/lzirondi/Github/snorkel/Scripts/Datasets/Processed/Candidatos.txt', 'w', encoding='UTF-8')
#for span in cands:
#    print(str(span))


In [ ]:
def liwcInCandidate(c):
    return 1 if se.contains(c.get_parent().words) else 0

    

for c in session.query(relation).filter(relation.split == 1).all():
    if liwcInCandidate(c) != 0:
        labeled.append(c)
print("Number labeled:", len(labeled))

In [ ]:
from snorkel.viewer import SentenceNgramViewer


#SentenceNgramViewer(labeled, session)


cands = session.query(relation).filter(relation.split == 0).all()

print(cands[0])
SentenceNgramViewer(cands, session)

#c = cands[0]

#print(c.get_parent().words)